In [ ]:
from glob import glob
import os 
import matplotlib
import matplotlib.colors
import matplotlib.pyplot as plt
import matplotlib.patheffects as PathEffects
import matplotlib.gridspec as gridspec
from scipy import signal
import pandas as pd
import numpy as np
import datetime


import plotly.graph_objects as go

# Read in the time series files of the power

In [ ]:
matplotlib.rcParams['font.family'] = 'sans-serif'
matplotlib.rcParams['font.sans-serif'] = ['Helvetica']
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
matplotlib.rcParams['font.size'] = 17

In [ ]:
def getPowerLog(path):
    with open(path) as f:
        data = f.readlines()

    datestring = ''.join(path.split(',')[-2:]).split('.')[0][1:]
    datetime_object = datetime.datetime.strptime(datestring, '%B %d %Y')
    datetime_object = datetime_object.replace(hour=11, minute=59)
    
    result = []; seconds = np.arange(0,60)
    for line in data:
        
        line = line.replace('z', '0')
        line = line.strip().replace('\ufeff', '').split(',')
        line_head = line[:2]

        del line[:2]
        del line[-1]

        datetime_object = datetime_object.replace(hour=int(line_head[0]), 
                                                  minute=int(line_head[1]))

        result += [(datetime_object.replace(second = seconds[index]), 
                         float(bit)) for index, bit in enumerate(line)]
    result = pd.DataFrame(result)
    result.columns = ['datetime', 'voltage']
    return result 

In [ ]:
eaton_0 = pd.read_csv('Eaton_0.csv')
eaton_1 = pd.read_csv('Eaton_1.csv')

fig = go.Figure([   go.Scatter(x=eaton_1.X, y=eaton_1.CH1), 
                    go.Scatter(x=eaton_0.X, y=eaton_0.CH1) ,       
                ])
fig.update_layout(template="plotly_dark")
fig.show()

In [ ]:
fs = 1 / (eaton_1.X[1:].astype('float')[2] - eaton_1.X[1:].astype('float')[1])

In [ ]:
plt.figure(figsize = (13, 5))
plt.subplot(121)
plt.plot(eaton_1.X[1:].astype('float'), eaton_1.CH1[1:].astype('float'), c = 'r', 
         label = 'Through Eaton 9355')
plt.plot(eaton_0.X[1:].astype('float'), eaton_0.CH1[1:].astype('float'), c = 'g', 
         label = 'Bypass Eaton 9355')
plt.ylim(-6,6)
plt.xlabel('relative time (s)')
plt.ylabel('voltage (V)')
plt.legend(frameon=False)

plt.title('Time Domain')




plt.subplot(122)
f, Pxx_den = signal.periodogram(eaton_1.CH1[1:].astype('float'), fs)
plt.semilogy(f, Pxx_den,  c = 'r', label = 'Through Eaton 9355')

f, Pxx_den = signal.periodogram(eaton_0.CH1[1:].astype('float'), fs)
plt.semilogy(f, Pxx_den, c = 'g', label = 'Bypass Eaton 9355')

plt.ylim([1e-6, 1e2])
plt.xlabel('frequency (Hz)')
plt.ylabel('Power Spectral Denstiy ($V^2$/Hz)')
plt.legend(frameon=False)
plt.title('Frequency Domain')

plt.suptitle('Comparsion observed at UP1A 6, NPL1', y = 1.05)
plt.tight_layout()
plt.savefig('Eaton 9355.pdf', transparent = True, dip = 300)
plt.show()

## The master dataframe df for the voltage values each second

In [ ]:
%%time
df = pd.concat([getPowerLog(file) for file in glob('*.dlg')])
df = df.sort_values(by=['datetime'])

In [ ]:
df.to_csv('powerlog.csv', index = False)

In [ ]:
fig = go.Figure([go.Scatter(x=df[-8000:].datetime, y=df[-8000:].voltage)])
fig.update_layout(template="plotly_dark")
fig.show()

In [ ]:
#fig = go.Figure([go.Scatter(x=df.datetime, y=df.voltage)])
#fig.update_layout(template="plotly_dark")
#fig.show()

In [ ]:
plt.figure(figsize=(15, 5))
plt.plot(df[-8000:].datetime, df[-8000:].voltage, marker = 'd', lw = 0.1,  markersize = 2, c = 'red')

plt.savefig('summary.pdf', dpi = 300)

# Reading the event log files .drp

In [ ]:
log = []
for file in glob('*.drp'):
    with open(file) as f:
        data = f.readlines()
        log += data
log = [line.replace('\ufeff', '').strip() for line in log]

log_start = [line for line in log if "start" in line]
log_start = [datetime.datetime.strptime(line.split('day, ')[-1], '%B %d, %Y, %I:%M:%S %p') for line in log_start]

log_end = [line for line in log if "stop" in line]
log_end = [datetime.datetime.strptime(line.split('day, ')[-1], '%B %d, %Y, %I:%M:%S %p') for line in log_end]

log_event = [line for line in log if "Event" in line]
log_event = [(datetime.datetime.strptime(''.join(line.split(',')[:2]), '%m-%d-%Y %I:%M:%S %p'), 
                  line.split('-')[-1].replace('>', '').strip() ) \
                     for line in log_event]

log_event = pd.DataFrame(log_event)
log_event.columns = ['datetime', 'event']

In [ ]:
log_start

In [ ]:
log_end

In [ ]:
log_event = log_event.sort_values(by = 'datetime')
log_event.to_csv('eventlog.csv', index = False)

In [ ]:
log_event